<a href="https://colab.research.google.com/github/aeDeaf/EasyEngine/blob/master/exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install physlearn==1.2.0

In [0]:
#import sys
#sys.path.append('/home/andrey/programming/Projects/PycharmProjects/physlearn/')
from physlearn.NeuralNet.NeuralNetPro import NeuralNetPro
from physlearn.NeuralNet.NeuralNet import NeuralNet
from physlearn.Optimizer.NelderMead import NelderMead
from physlearn.Optimizer.DifferentialEvolution import DifferentialEvolution
from IPython.display import clear_output
import numpy
import matplotlib.pylab as plt
from math import sqrt

from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


In [0]:
def scalar_dot(x, y):
    res = 0
    for index, _ in enumerate(x):
        res += x[index] * y[index]
    return res

In [0]:
def norm(x):
    return sqrt(scalar_dot(x, x))

In [0]:
def gramm_shmit(matrix):
    new_matrix = numpy.zeros_like(matrix)
    for i in range(matrix.shape[0]):
        new_vector = matrix[i]
        for j in range(i):
            index = i - j - 1
            new_vector = new_vector - scalar_dot(matrix[i], new_matrix[index]) * new_matrix[index]
        new_matrix[i] = new_vector / norm(new_vector)
    return new_matrix

In [0]:
def create_V_matrix(N):
    rank = N + 1
    while rank != N:
        V = numpy.random.uniform(-50, 50, (N, N))
        rank = numpy.linalg.matrix_rank(V)
    return gramm_shmit(V)

In [0]:
def create_matrix(N):
    diag_matrix = numpy.zeros((N, N))
    diag_elements = numpy.random.uniform(1, 10, N)
    for index, item in enumerate(diag_elements):
        diag_matrix[index][index] = item
    V = create_V_matrix(N)
    matrix = numpy.matmul(numpy.matmul(V, diag_matrix), numpy.transpose(V))
    return matrix, diag_matrix

In [0]:
def gaussian(x, A):
    inner_part = numpy.matmul(numpy.matmul(numpy.transpose(x), A), x)
    return numpy.exp(-inner_part)

In [0]:
min_element = -0.7
max_element = 0.7
amount_of_points = 8

In [0]:
max_dimension = 6

In [0]:
def make_all_variants(A):
    N = A.shape[0]
    amount_of_points = A.shape[1]
    res  = numpy.zeros((N, amount_of_points ** N))
    for i in range(N):
        for index, item in enumerate(A[i]):
            for j in range(i + 2):
                left_break = (amount_of_points ** (N - i - 1)) * index + j * amount_of_points ** (N - i)
                right_break = (amount_of_points ** (N - i - 1)) * (index + 1) + j * amount_of_points ** (N - i) 
                res[i, left_break:right_break] = item
    return res

In [0]:
def make_data(N):
    x = numpy.empty((N, amount_of_points))
    for i in range(N):
        x[i] = numpy.linspace(min_element, max_element, amount_of_points)
    x_train = make_all_variants(x)
    return x_train

In [0]:
def make_y_train(x_train, N):
    y_train = []
    A = create_matrix(N)[0]
    for j in range(amount_of_points ** N):
        x = []
        for i in range(N):
            x.append([x_train[i][j]])
        y_train.append(gaussian(x, A))
    return numpy.array(y_train).reshape(1, amount_of_points ** N)

In [0]:
x_train = make_data(1)

In [0]:
y_train = make_y_train(x_train, 1)

In [0]:
def create_net(amount_of_hidden_units, N):
    net = NeuralNetPro()
    net.add_input_layer(N)
    net.add(amount_of_hidden_units, net.sigmoid)
    net.add_output_layer(1, net.linear)
    net.compile()
    net.set_train_type('prediction')
    net.set_random_matrixes()
    dim = net.return_unroll_dim()
    return net, dim

In [0]:
nm = NelderMead()

In [0]:
net, dim = create_net(10, 1)

def cost(params):
    net.roll_matrixes(params)
    return net.calculate_cost(x_train, y_train)

def find_optimal_aohu(start_aohu, N):
    print('Dimension:', N)
    amount_of_hidden_units = start_aohu
    exit_code = -1
    cost_function = -1
    global net
    #net, dim = create_net(amount_of_hidden_units)
    while exit_code != 0:
        del net
        net, dim = create_net(amount_of_hidden_units, N)
        res = nm.optimize(cost, dim, 600000, alpha=0.35, min_cost=2e-4)
        exit_code = res.exit_code
        cost_function = res.cost_function
        clear_output()
        print('Dimension:', N)
        print('----------------------------')
        print('Amount of hidden units:', amount_of_hidden_units)
        print('----------------------------')
        print(res)
        if exit_code != 0:
            amount_of_hidden_units += 1
    return amount_of_hidden_units

In [0]:
optimal_aohu_list = [] 
start_aohu = 8
for N in range(5, max_dimension + 1):
    x_train = make_data(N)
    y_train = make_y_train(x_train, N)
    res = find_optimal_aohu(start_aohu, N)
    optimal_aohu_list.append(res)
    start_aohu = res

Dimension: 5
----------------------------
Amount of hidden units: 9
----------------------------
Is converge: True
Amount of iterations: 121974
Total time: 317.57 s
Reached function value: 0.0007385268269212551
Reason of break: Local minimum reached



...  178353 (29%) 132.868 it\s

KeyboardInterrupt: ignored

In [0]:
plt.plot(optimal_aohu_list, linestyle='--', marker='o')